In [ ]:
## PARÁMETROS BÁSICOS

traits_file          <- "../traits/traits_low-level.csv"  # CSV con 'species' + rasgos
mcc_tree_file        <- "../trees/BEAST_MCC.tre"          # árbol MCC (simple)
posterior_trees_file <- "../trees/BEAST_posterior.nex"    # posterior (multiPhylo)
trait_cols           <- c("A", "SF", "H", "TC", "SC")    # nombres de columnas de rasgos
n_posterior_trees    <- 100                               # nº de árboles del posterior
out_dir              <- "../results"                         # carpeta de salida

## LIBRERÍAS

library(ape)
library(phytools)

if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE)

## LEER DATOS

traits <- read.csv(traits_file, stringsAsFactors = FALSE)

# Nos aseguramos que la columna de especies se llame 'species'
colnames(traits)[1] <- "species"
traits$species <- as.character(traits$species)

# Leer árbol MCC
tree_mcc <- read.tree(mcc_tree_file)

# Leer árboles del posterior y quedarnos con los primeros n_posterior_trees
trees_posterior <- read.nexus(posterior_trees_file)
trees_posterior <- trees_posterior[1:n_posterior_trees]
n_trees <- length(trees_posterior)


## FUNCIÓN: SEÑAL PARA 1 RASGO EN 1 ÁRBOL

phylo_signal_one <- function(tree, traits_df, trait_name) {
  x <- traits_df[[trait_name]]
  names(x) <- traits_df$species
  x <- x[!is.na(x)]  # quitamos posibles NA
  
  k_res <- phylosig(tree, x, method = "K",      test = TRUE)
  l_res <- phylosig(tree, x, method = "lambda", test = TRUE)
  
  c(
    K        = k_res$K,
    K_p      = k_res$P,
    lambda   = l_res$lambda,
    lambda_p = l_res$P
  )
}

## MCC: SEÑAL FILOGENÉTICA PARA TODOS LOS RASGOS

res_mcc <- data.frame()

for (tr in trait_cols) {
  vals <- phylo_signal_one(tree_mcc, traits, tr)
  
  res_mcc <- rbind(
    res_mcc,
    data.frame(
      trait    = tr,
      K        = vals["K"],
      K_p      = vals["K_p"],
      lambda   = vals["lambda"],
      lambda_p = vals["lambda_p"],
      row.names = NULL
    )
  )
}

# Guardar resultados MCC
write.csv(res_mcc,
          file.path(out_dir, "phylo_signal_MCC.csv"),
          row.names = FALSE)


## POSTERIOR: SEÑAL PARA TODOS LOS RASGOS EN TODOS LOS ÁRBOLES


res_post <- data.frame()

for (i in 1:n_trees) {
  tree_i <- trees_posterior[[i]]
  
  for (tr in trait_cols) {
    vals <- phylo_signal_one(tree_i, traits, tr)
    
    res_post <- rbind(
      res_post,
      data.frame(
        tree_id  = i,
        trait    = tr,
        K        = vals["K"],
        K_p      = vals["K_p"],
        lambda   = vals["lambda"],
        lambda_p = vals["lambda_p"],
        row.names = NULL
      )
    )
  }
}

# Guardar resultados posterior
write.csv(res_post,
          file.path(out_dir, "phylo_signal_posterior.csv"),
          row.names = FALSE)

### GRAFICOS DE VIOLIN

In [ ]:

## VIOLIN PLOTS PARA λ Y K (MCC + POSTERIOR)

library(dplyr)
library(ggplot2)
library(rlang)   # para sym()
library(grid)    # para unit() en márgenes/espaciados

## PARÁMETROS 

## Leyenda
source_levels <- c("MCC", "Posterior")
source_labels <- c(
  MCC       = "MCC",
  Posterior = "Posterior sample (100 trees)"
)

legend_position_inside      <- c(0.35, 0.97)  # posición dentro del panel (x, y)
legend_justification_inside <- c(1, 1)        # ancla del recuadro (1,1 = esquina sup. der.)
legend_text_size            <- 12              # texto leyenda
legend_box_linewidth        <- 0.4

#leyenda
legend_margin     <- margin(2, 2, 2, 2)
legend_spacing_y  <- unit(0.2, "lines")
legend_key_size   <- unit(0.4, "lines")

## Apariencia de violines y puntos
fill_values <- c(
  MCC       = "#1f78b4",  # azul puntos MCC
  Posterior = "grey70"    # gris violines posterior
)

violin_width    <- 0.8
violin_alpha    <- 0.6
violin_line_col <- "grey40"
violin_line_lwd <- 0.4

point_shape      <- 21
point_border_col <- "black"
point_size       <- 3

## Ejes y texto
y_break      <- 0.2
base_font    <- 16
axis_text_sz <- 14

## Tamaño de salida
fig_width  <- 8
fig_height <- 6
fig_dpi    <- 300

# CONSTRUIR DATA FRAMES λ Y K

lambda_df <- bind_rows(
  res_mcc %>%
    transmute(trait, lambda, source = "MCC"),
  res_post %>%
    transmute(trait, lambda, source = "Posterior")
)

K_df <- bind_rows(
  res_mcc %>%
    transmute(trait, K, source = "MCC"),
  res_post %>%
    transmute(trait, K, source = "Posterior")
)

## Orden de rasgos y niveles de 'source'
lambda_df <- lambda_df %>%
  mutate(
    trait  = factor(trait, levels = trait_cols),
    source = factor(source, levels = source_levels)
  )

K_df <- K_df %>%
  mutate(
    trait  = factor(trait, levels = trait_cols),
    source = factor(source, levels = source_levels)
  )

## REGLA ÚNICA PARA LÍMITES DE EJE Y

y_round_top     <- 0.1
lambda_min_top  <- 1.1
K_min_top       <- 1.0

calc_ymax <- function(x, round_to = 0.1, min_top = 1.0){
  mx <- max(x, na.rm = TRUE)
  ymax <- ceiling(mx / round_to) * round_to
  if (ymax < min_top) ymax <- min_top
  ymax
}

max_lambda <- calc_ymax(lambda_df$lambda, round_to = y_round_top, min_top = lambda_min_top)
max_K      <- calc_ymax(K_df$K,       round_to = y_round_top, min_top = K_min_top)

## FUNCIÓN GENÉRICA PARA HACER EL VIOLIN PLOT

make_violin_plot <- function(df, y_col, y_label, y_max, file_name) {

  gg <- ggplot(df, aes(x = trait, y = !!sym(y_col), fill = source)) +
    # violines: posterior
    geom_violin(
      data      = subset(df, source == "Posterior"),
      width     = violin_width,
      alpha     = violin_alpha,
      colour    = violin_line_col,
      linewidth = violin_line_lwd
    ) +
    # puntos: MCC
    geom_point(
      data   = subset(df, source == "MCC"),
      shape  = point_shape,
      colour = point_border_col,
      size   = point_size
    ) +
    scale_fill_manual(
      name   = NULL,
      values = fill_values,
      breaks = source_levels,
      labels = source_labels
    ) +
    guides(
      fill = guide_legend(
        override.aes = list(
          shape  = c(point_shape, 22),
          colour = point_border_col
        )
      )
    ) +
    scale_y_continuous(
      limits = c(0, y_max),
      breaks = seq(0, y_max, by = y_break),
      expand = expansion(mult = c(0, 0))
    ) +
    labs(x = "Trait (low-level feature)", y = y_label) +
    theme_bw(base_size = base_font) +
    theme(
      panel.grid.major.x = element_line(colour = "grey85", linewidth = 0.5),
      panel.grid.major.y = element_line(colour = "grey85", linewidth = 0.5),
      panel.grid.minor.x = element_blank(),
      panel.grid.minor.y = element_line(colour = "grey92", linewidth = 0.3),

      legend.position             = "inside",
      legend.position.inside      = legend_position_inside,
      legend.justification.inside = legend_justification_inside,
      legend.title                = element_blank(),
      legend.text                 = element_text(size = legend_text_size),
      legend.background           = element_rect(
        fill      = "white",
        colour    = "black",
        linewidth = legend_box_linewidth
      ),
      legend.margin     = legend_margin,
      legend.spacing.y  = legend_spacing_y,
      legend.key.size   = legend_key_size,
      legend.key        = element_rect(fill = NA, colour = NA),
      axis.text  = element_text(size = axis_text_sz),
      axis.title = element_text(size = base_font)
    )

  ggsave(
    filename = file.path(out_dir, file_name),
    plot     = gg,
    width    = fig_width,
    height   = fig_height,
    dpi      = fig_dpi,
    bg       = "white"
  )

  invisible(gg)
}

## GENERAR Y GUARDAR LAS FIGURAS

plot_lambda <- make_violin_plot(
  df        = lambda_df,
  y_col     = "lambda",
  y_label   = "Pagel's \u03BB",
  y_max     = max_lambda,
  file_name = "fig_lambda_violin.png"
)

plot_K <- make_violin_plot(
  df        = K_df,
  y_col     = "K",
  y_label   = "Blomberg's K",
  y_max     = max_K,
  file_name = "fig_K_violin.png"
)


### RESUMEN ESTADÍSTICO

In [ ]:
## RESUMEN λ y K (MCC + POSTERIOR) POR RASGO
## - Mediana + IC 95% (2.5%–97.5%)


library(dplyr)

summary_table <- res_post %>%
  group_by(trait) %>%
  summarise(
    # Posterior Pagel's λ
    lambda_med  = median(lambda, na.rm = TRUE),
    lambda_low  = quantile(lambda, 0.025, na.rm = TRUE),
    lambda_high = quantile(lambda, 0.975, na.rm = TRUE),
    # Posterior Blomberg's K
    K_med       = median(K, na.rm = TRUE),
    K_low       = quantile(K, 0.025, na.rm = TRUE),
    K_high      = quantile(K, 0.975, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  # Añadir valores MCC
  left_join(
    res_mcc %>%
      select(
        trait,
        lambda_MCC = lambda,
        K_MCC      = K
      ),
    by = "trait"
  ) %>%
  # Ordenar columnas
  select(
    trait,
    lambda_MCC, lambda_med, lambda_low, lambda_high,
    K_MCC,      K_med,      K_low,      K_high
  ) %>%
  arrange(trait) %>%
  mutate(
    across(
      where(is.numeric),
      ~ round(.x, 3)
    )
  )

print(summary_table, width = Inf)

write.csv(summary_table, file = "../results/summary_phylo_signal.csv", row.names = FALSE)



In [ ]:
## RESUMEN TESTS DE phytools (p-valores λ y K)

library(dplyr)

# Función de formato
fmt_smart <- function(x,
                      dec = 3,
                      sci_lower = 1e-3,
                      sci_upper = 1e3) {
  sapply(x, function(v) {
    if (is.na(v)) return(NA_character_)
    
    # Cero va siempre en formato fijo
    if (v == 0) {
      return(sprintf(paste0("%.", dec, "f"), v))
    }
    
    if (abs(v) >= sci_lower && abs(v) < sci_upper) {
      # Rango "normal": decimal fijo
      sprintf(paste0("%.", dec, "f"), v)
    } else {
      # Muy pequeño o muy grande: notación científica
      sprintf(paste0("%.", dec, "e"), v)
    }
  })
}

tests_summary <- res_post %>%
  group_by(trait) %>%
  summarise(
    # Posterior Pagel's lambda (p-values)
    lambda_p_min = min(lambda_p, na.rm = TRUE),
    lambda_p_med = median(lambda_p, na.rm = TRUE),
    lambda_p_max = max(lambda_p, na.rm = TRUE),
    # Posterior Blomberg's K (p-values)
    K_p_min      = min(K_p, na.rm = TRUE),
    K_p_med      = median(K_p, na.rm = TRUE),
    K_p_max      = max(K_p, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  # Añadir p-valores del MCC
  left_join(
    res_mcc %>%
      select(
        trait,
        lambda_p_MCC = lambda_p,
        K_p_MCC      = K_p
      ),
    by = "trait"
  ) %>%
  # Ordenar columnas
  select(
    trait,
    lambda_p_MCC, lambda_p_min, lambda_p_med, lambda_p_max,
    K_p_MCC,      K_p_min,      K_p_med,      K_p_max
  ) %>%
  arrange(trait) %>%
  # Aplicar formato inteligente a todas las columnas numéricas
  mutate(across(
    where(is.numeric),
    ~ fmt_smart(.x, dec = 3)
  ))

print(tests_summary, width = Inf)

## Guardar CSV ya formateado
write.csv(
  tests_summary,
  file = "../results/summary_phylo_tests.csv",
  row.names = FALSE
)
